# This is the development of the Model
<hr>

- based on the llamaindex framework
- Vector database is chromaDB

Set-up the enviroment

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')